## Raspberry Pi and Arduino Ultrasonic Snow Depth Sensor
Carter Humphreys, 11/02/2020

[github.com/HumphreysCarter](https://github.com/HumphreysCarter/Ultrasonic-Snow-Depth-Sensor)

In [160]:
import math
import serial
import numpy as np
import pandas as pd
from datetime import datetime
from metpy.units import units

In [ ]:
# Start serial connection with sensor
sensor = serial.Serial('/dev/ttyACM0', 9600, 8, 'N', 1, timeout=1)

# Initialize empty dataframe
records=pd.DataFrame()

# Read in new data until 10 records are recorded
while len(records) < 10:
    
    # Read data from sensor
    record = sensor.readline()
    
    # De-code byte object as string
    record = record.decode("utf-8")
       
    # Check if valid record
    if '!***' in record and '***!' in record:
              
        # Remove start/end strings and charcaters
        record = record.replace('!*** ', '')
        record = record.replace(' ***!', '')
        record = record.replace('\r\n', '')
        
        # Convert to int
        record = int(record)
               
        # Save record
        records = records.append([{'DateTime_UTC':datetime.utcnow(), 'WaveSpeed_μs':record}], ignore_index=True)
        
# Close serial connection with sensor
sensor.close()

In [149]:
records

,DateTime_UTC,WaveSpeed_μs
0,2020-11-02 22:30:03.116861,7270
1,2020-11-02 22:30:03.125975,7220
2,2020-11-02 22:30:03.141289,7247
3,2020-11-02 22:30:03.155014,7249
4,2020-11-02 22:30:03.168444,7247
5,2020-11-02 22:30:03.181976,7272
6,2020-11-02 22:30:03.195226,7297
7,2020-11-02 22:30:03.208920,7218
8,2020-11-02 22:30:03.222438,7270
9,2020-11-02 22:30:03.235558,7220


In [184]:
def calc_speed_of_sound(T):
    """
    Calculates speed of sound based on air temperature
    """
    # Constants
    k = 1.4
    R = 286.9
    T = T.to('kelvin').magnitude
    
    # Speed of sound equation    
    c = math.pow(1.4*(R*T), 0.5) * units('m/s')
    
    return c

def calc_snow_depth(sensor_height, wave_speed, speed_of_sound):
    """
    Calculates the snow depth from the total trip time of the sound wave
    """
    # Convert units for calculation
    sensor_height = sensor_height.to('m')
    wave_speed = wave_speed.to('s')
    speed_of_sound = speed_of_sound.to('m/s')
    
    # Divide total wave speed by 2 to get inital trip speed
    wave_speed = wave_speed / 2
    
    # Get distance from sensor to surface
    d = wave_speed * speed_of_sound
    
    # Subtract depth from sensor height to get snow depth
    snow_depth = sensor_height - d
    
    return snow_depth
    
def process_sensor_data(records, sensor_height, air_temp, unit='cm', precision=1, truncate_neg_zero=True, no_negative=False):
    """
    Gets snow depth from sensor records
    """
    # Calculate speed of sound from air temperature
    speed_of_sound = calc_speed_of_sound(air_temp)
    
    # Calcualte snow depth from each record
    snow_depth_obs = []
    for wave_speed in records.WaveSpeed_μs:
        # Attach units to wave speed
        wave_speed = wave_speed * units.microseconds

        # Calculate snow depth
        snow_depth = calc_snow_depth(sensor_height, wave_speed, speed_of_sound)
        snow_depth_obs.append(snow_depth)

    # Calculate average depth from observations
    snow_depth = sum(snow_depth_obs)/len(snow_depth_obs)

    # Round to nearest tenth of a centimeter
    snow_depth = round(snow_depth.to(unit), precision)
    
    # Check if number is negative
    if no_negative and snow_depth.magnitude < 0:
        return (records.DateTime_UTC[0], np.nan)
    
    # Change values between -1 and 0 to 0
    if truncate_neg_zero and snow_depth.magnitude <= 0 and snow_depth.magnitude > -1:
        return (records.DateTime_UTC[0], round(0.0  * units(unit), precision))
    
    return (records.DateTime_UTC[0], snow_depth)

In [185]:
# Specify sensor height
sensor_height = 120.3 * units.cm

# Specify air temeprature
air_temp = 34 * units.degF

# Process sensor observations
time, snow_depth = process_sensor_data(records, sensor_height, air_temp)
snow_depth

0.0 <Unit('centimeter')>

In [187]:
# Export to file
with open('snow_depth/snow_depth.csv', 'a') as f:
    f.write(f'\n{time},{snow_depth.magnitude},{snow_depth.units}')